In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# 공통 요청 파라미터 설정
KEY = '7a50474368636f64363463436c436f'
TYPE = 'json'
START_INDEX = 1
END_INDEX = 1000

# 시작 날짜 설정
start_date = datetime.strptime('2020-06-01', '%Y-%m-%d').date()

# 오늘 날짜까지의 데이터 가져오기
end_date = datetime.now().date()

# 주차장 데이터 가져오기
SERVICE = 'TbUseMonthstatusView'

# 데이터를 저장할 빈 DataFrame 생성
parking_dfs = []

# API 호출을 위한 URL 생성 함수
def create_url(service, date):
    return f"http://openapi.seoul.go.kr:8088/{KEY}/{TYPE}/{service}/{START_INDEX}/{END_INDEX}/{date}/"

# 주차장 데이터 수집 함수
def retrieve_parking_data(date):
    url = create_url(SERVICE, date)
    response = requests.get(url)
    data = response.json()
    try:
        items = data[SERVICE]['row']
        parking_dfs.append(pd.DataFrame(items))
    except KeyError:
        print(f"No parking data available for date: {date}")

# 날짜별 주차장 데이터 수집 실행
while start_date <= end_date:
    date_str = start_date.strftime('%Y%m%d')
    retrieve_parking_data(date_str)
    start_date += timedelta(days=1)

# 모든 DataFrame을 병합
if parking_dfs:
    df = pd.concat(parking_dfs)

    # 필요한 컬럼 선택
    selected_df = df[['DT', 'USE_CNT', 'USE_MIN']]

    # 날짜별로 USE_CNT와 USE_MIN 합치기
    aggregated_df = selected_df.groupby('DT').sum().reset_index()

    # 총합 계산
    total_cnt = aggregated_df['USE_CNT'].sum()
    total_min = aggregated_df['USE_MIN'].sum()


    # 상대적 비율 계산 및 소수점 2자리까지 반올림
    aggregated_df['USE_CNT_RATIO'] = round((aggregated_df['USE_CNT'] / total_cnt) * 2000, 2)
    aggregated_df['USE_MIN_RATIO'] = round((aggregated_df['USE_MIN'] / total_min) * 2000, 2)

    # CSV 파일 저장
    save_path = 'C:\\Users\\user\\Desktop\\2023-1\\data-analysis\\aggregated_parking_data.csv'
    aggregated_df.to_csv(save_path, index=False)

    print("Aggregated parking data saved.")
else:
    print("No parking data available for the specified period.")
